In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
bbbp_chemberta2_valid2 = pd.read_csv('./chemberta2/results/bbbp/chemberta2_valid2_bbbp_3_predictions.csv')
bbbp_molformer_valid2 = pd.read_csv('./molformer/results/bbbp/molformer_valid2_bbbp_3_epoch29.csv')
bbbp_molbert_valid2 = pd.read_csv('./molbert/results/bbbp/molbert_valid2_bbbp_3.csv')

# Load the test data for each model
bbbp_chemberta2_test = pd.read_csv('./chemberta2/results/bbbp/chemberta2_test_bbbp_3_predictions.csv')
bbbp_molformer_test = pd.read_csv('./molformer/results/bbbp/molformer_test_bbbp_3_epoch29.csv')
bbbp_molbert_test = pd.read_csv('./molbert/results/bbbp/molbert_test_bbbp_3.csv')

# features

# Load the features from chemberta
bbbp_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/bbbp/chemberta2_valid2_bbbp_3_features.csv')
bbbp_chemberta2_features_test = pd.read_csv('./chemberta2/features/bbbp/chemberta2_test_bbbp_3_features.csv')

# Load the features from molformer
bbbp_molformer_features_valid2 = pd.read_csv('./molformer/features/bbbp/molformer_valid2_bbbp_3_features.csv')
bbbp_molformer_features_test = pd.read_csv('./molformer/features/bbbp/molformer_test_bbbp_3_features.csv')

# Load the features from molbert
bbbp_molbert_features_valid2 = pd.read_csv('./molbert/features/bbbp/molbert_valid2_bbbp_3_features.csv')
bbbp_molbert_features_test = pd.read_csv('./molbert/features/bbbp/molbert_test_bbbp_3_features.csv')

In [2]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Preparing the actual and predicted values
# Chemberta2
bbbp_chemberta_actual = bbbp_chemberta2_test['p_np']
bbbp_chemberta_pred = bbbp_chemberta2_test['y_pred']
bbbp_chemberta_probs = bbbp_chemberta2_test[['softmax_class_0_prob', 'softmax_class_1_prob']]

# Molformer
bbbp_molformer_actual = bbbp_molformer_test['Actual']
bbbp_molformer_pred = (bbbp_molformer_test['Prob_Class_1'] > 0.5).astype(int)
bbbp_molformer_probs = bbbp_molformer_test[['Prob_Class_0', 'Prob_Class_1']]

# Molbert
bbbp_molbert_actual = bbbp_molbert_test['target']
bbbp_molbert_pred = bbbp_molbert_test['pred']
bbbp_molbert_probs = bbbp_molbert_test['prob']

# Calculating metrics
bbbp_metrics_results = {}

for model_name, actual, pred, probs in [("Chemberta2", bbbp_chemberta_actual, bbbp_chemberta_pred, bbbp_chemberta_probs['softmax_class_1_prob']),
                                         ("Molformer", bbbp_molformer_actual, bbbp_molformer_pred, bbbp_molformer_probs['Prob_Class_1']),
                                         ("Molbert", bbbp_molbert_actual, bbbp_molbert_pred, bbbp_molbert_probs)]:
    bbbp_metrics_results[model_name] = {
        "Accuracy": accuracy_score(actual, pred),
        "F1 Score": f1_score(actual, pred),
        "ROC-AUC": roc_auc_score(actual, probs),
        "PR-AUC": average_precision_score(actual, probs)
    }

bbbp_metrics_results

{'Chemberta2': {'Accuracy': 0.8292682926829268,
  'F1 Score': 0.8844884488448845,
  'ROC-AUC': 0.9488724127278345,
  'PR-AUC': 0.9882303014277589},
 'Molformer': {'Accuracy': 0.8774509803921569,
  'F1 Score': 0.924924924924925,
  'ROC-AUC': 0.8789632213062777,
  'PR-AUC': 0.965741487133955},
 'Molbert': {'Accuracy': 0.9068627450980392,
  'F1 Score': 0.9425981873111783,
  'ROC-AUC': 0.9514901712111604,
  'PR-AUC': 0.9885586344415213}}

In [3]:
# Identify the 'smiles' values in chemberta2_valid2 that are not in molbert_valid2
missing_smiles_molformer_valid2 = set(bbbp_chemberta2_valid2['smiles']) - set(bbbp_molformer_valid2['smiles'])
print(f"Missing smiles in molformer_valid2: {missing_smiles_molformer_valid2}")

# Identify the 'smiles' values in chemberta2_valid2 that are not in molbert_valid2
missing_smiles_molbert_valid2 = set(bbbp_chemberta2_valid2['smiles']) - set(bbbp_molbert_valid2['smiles'])
print(f"Missing smiles in molbert_valid2: {missing_smiles_molbert_valid2}")

# Combine the invalid indices from molformer_valid2 with the missing indices from molbert_valid2
combined_invalid_smiles_valid2 = list(set(missing_smiles_molformer_valid2).union(set(missing_smiles_molbert_valid2)))

print(f"These indices will be removed from the valid2 set: {combined_invalid_smiles_valid2}")

Missing smiles in molformer_valid2: {'c1c(c(ncc1)CSCCN\\C(=[NH]\\C#N)NCC)Br', 's1cc(nc1\\[NH]=C(\\N)N)C', 's1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)N', 'c1(nc(NC(N)=[NH2])sc1)CSCCNC(=[NH]C#N)NC', 'n1c(csc1\\[NH]=C(\\N)N)c1ccccc1'}
Missing smiles in molbert_valid2: {'[Na+].Cc1sc(SCC2=C(N3[C@H](SC2)[C@H](NC(=O)Cn4cnnn4)C3=O)C([O-])=O)nn1', 'c1c(c(ncc1)CSCCN\\C(=[NH]\\C#N)NCC)Br', '[Na+].CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](NC(=O)C3=CC=C(NC3=O)c4ccc(cc4)[S](=O)(=O)N(CCO)CCO)c5ccc(O)cc5)C(=O)N2[C@H]1C([O-])=O', '[Na+].Cn1nnnc1SCC2=C(N3[C@H](SC2)[C@H](NC(=O)CSC(F)(F)F)C3=O)C([O-])=O', 's1cc(nc1\\[NH]=C(\\N)N)C', 's1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)N', 'c1(nc(NC(N)=[NH2])sc1)CSCCNC(=[NH]C#N)NC', 'n1c(csc1\\[NH]=C(\\N)N)c1ccccc1', '[Na+].CO\\N=C(C(=O)N[C@@H]1[C@@H]2SCC(=C(N2C1=O)C([O-])=O)COC(C)=O)\\c3csc(N)n3'}
These indices will be removed from the valid2 set: ['[Na+].Cc1sc(SCC2=C(N3[C@H](SC2)[C@H](NC(=O)Cn4cnnn4)C3=O)C([O-])=O)nn1', 'c1c(c(ncc1)CSCCN\\C(=[NH]\\C#N)NCC)Br', '[Na+].CC

In [4]:
# Function to remove invalid SMILES
def remove_invalid_smiles(df, invalid_smiles_list):
    return df[~df['smiles'].isin(invalid_smiles_list)]

# Remove invalid SMILES from each dataframe
bbbp_chemberta2_valid2 = remove_invalid_smiles(bbbp_chemberta2_valid2, combined_invalid_smiles_valid2)
bbbp_molformer_valid2 = remove_invalid_smiles(bbbp_molformer_valid2, combined_invalid_smiles_valid2)
bbbp_molbert_valid2 = remove_invalid_smiles(bbbp_molbert_valid2, combined_invalid_smiles_valid2)
bbbp_chemberta2_features_valid2 = remove_invalid_smiles(bbbp_chemberta2_features_valid2, combined_invalid_smiles_valid2)
bbbp_molformer_features_valid2 = remove_invalid_smiles(bbbp_molformer_features_valid2, combined_invalid_smiles_valid2)
bbbp_molbert_features_valid2 = remove_invalid_smiles(bbbp_molbert_features_valid2, combined_invalid_smiles_valid2)

# Print the shapes of the dataframes after removal
print(bbbp_chemberta2_valid2.shape)
print(bbbp_molformer_valid2.shape)
print(bbbp_molbert_valid2.shape)
print(bbbp_chemberta2_features_valid2.shape)
print(bbbp_molformer_features_valid2.shape)
print(bbbp_molbert_features_valid2.shape)

(401, 8)
(401, 6)
(401, 4)
(401, 386)
(401, 769)
(401, 769)


In [5]:
bbbp_y_ensemble_valid2 = bbbp_chemberta2_valid2['p_np']

# Convert the ensemble target to a Series if not already done
bbbp_y_ensemble_valid2_s = pd.Series(bbbp_y_ensemble_valid2).reset_index(drop=True)

# Create dataframes for each model's class 1 probability
bbbp_chemberta2_prob = pd.DataFrame({'chemberta2': bbbp_chemberta2_valid2['softmax_class_1_prob']})
bbbp_chemberta2_prob.reset_index(drop=True, inplace=True)

bbbp_molformer_prob = pd.DataFrame({'molformer': bbbp_molformer_valid2['Prob_Class_1']})
bbbp_molformer_prob.reset_index(drop=True, inplace=True)

bbbp_molbert_prob = pd.DataFrame({'molbert': bbbp_molbert_valid2['prob']})
bbbp_molbert_prob.reset_index(drop=True, inplace=True)

# do the same for features bbbp_chemberta2_features_valid2.iloc[:, 2:]
bbbp_chemberta2_features = pd.DataFrame(bbbp_chemberta2_features_valid2.iloc[:, 2:])
bbbp_chemberta2_features.reset_index(drop=True, inplace=True)

bbbp_molformer_features = pd.DataFrame(bbbp_molformer_features_valid2.iloc[:, 1:])
bbbp_molformer_features.reset_index(drop=True, inplace=True)

bbbp_molbert_features = pd.DataFrame(bbbp_molbert_features_valid2.iloc[:, 1:])
bbbp_molbert_features.reset_index(drop=True, inplace=True)

# bbbp_features = pd.concat([bbbp_chemberta2_features, bbbp_molformer_features, bbbp_molbert_features], axis=1)

# Combine probabilities into one dataframe
train_bbbp_prob = pd.concat([bbbp_chemberta2_prob, bbbp_molformer_prob, bbbp_molbert_prob], axis=1)

# Function to calculate BCE for each row with epsilon to avoid log(0)
def calculate_bce_rowwise(y_true, y_pred, epsilon=1e-12):
    # Clip y_pred to ensure it's between epsilon and 1-epsilon
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Calculate row-wise BCE for each model
bce_chemberta = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_chemberta2_prob['chemberta2'])
bce_molformer = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_molformer_prob['molformer'])
bce_molbert = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_molbert_prob['molbert'])

# Create a dataframe for row-wise BCE losses
bce_loss_df = pd.DataFrame({
    'bce_chemberta': bce_chemberta,
    'bce_molformer': bce_molformer,
    'bce_molbert': bce_molbert
})

# Final ensemble X matrix: Combine row-wise BCE losses, predictions, and features
bbbp_X_ensemble_valid2 = pd.concat([bce_loss_df, train_bbbp_prob], axis=1)

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Combine probabilities with their respective feature sets
chemberta_X = pd.concat([bbbp_chemberta2_prob, bbbp_chemberta2_features], axis=1)
molformer_X = pd.concat([bbbp_molformer_prob, bbbp_molformer_features], axis=1)
molbert_X = pd.concat([bbbp_molbert_prob, bbbp_molbert_features], axis=1)

# Standardize each dataset
scaler_chemberta = StandardScaler().fit(chemberta_X)
scaler_molformer = StandardScaler().fit(molformer_X)
scaler_molbert = StandardScaler().fit(molbert_X)

chemberta_X_scaled = scaler_chemberta.transform(chemberta_X)
molformer_X_scaled = scaler_molformer.transform(molformer_X)
molbert_X_scaled = scaler_molbert.transform(molbert_X)

# Define the binary cross-entropy loss values as target variables (y)
chemberta_y_bce = bce_chemberta  # Row-wise BCE loss calculated earlier
molformer_y_bce = bce_molformer  # Row-wise BCE loss calculated earlier
molbert_y_bce = bce_molbert      # Row-wise BCE loss calculated earlier

# Ensure that X_scaled and y_bce are numpy arrays
chemberta_X_scaled = np.array(chemberta_X_scaled)
molformer_X_scaled = np.array(molformer_X_scaled)
molbert_X_scaled = np.array(molbert_X_scaled)

chemberta_y_bce = np.array(chemberta_y_bce)
molformer_y_bce = np.array(molformer_y_bce)
molbert_y_bce = np.array(molbert_y_bce)

torch.manual_seed(0)
np.random.seed(0)

# Define custom RMSE loss function
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]  # Output is continuous, no activation function
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Objective function for Bayesian optimization
def objective(params, X, y):
    kf = KFold(n_splits=5)
    rmses = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        train_dataset = TensorDataset(torch.tensor(X_train.astype(np.float32)), 
                                      torch.tensor(y_train.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=int(params['num_layers']), 
                         num_neurons=int(params['num_neurons']), dropout_rate=params['dropout_rate'])
        criterion = RMSELoss()  # Use RMSELoss for regression
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):  # Set number of epochs
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val.astype(np.float32))
            y_val_tensor = torch.tensor(y_val.astype(np.float32)).unsqueeze(-1)
            outputs = model(X_val_tensor)
            rmse = np.sqrt(mean_squared_error(y_val_tensor.numpy(), outputs.numpy()))
            rmses.append(rmse)

    avg_rmse = np.mean(rmses)
    return {'loss': avg_rmse, 'status': STATUS_OK}  # Minimize RMSE

# Hyperparameter space for optimization
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

# Scaled input data (after standardization) and corresponding target (binary cross-entropy loss)
datasets = {
    'chemberta': (chemberta_X_scaled, chemberta_y_bce),
    'molformer': (molformer_X_scaled, molformer_y_bce),
    'molbert': (molbert_X_scaled, molbert_y_bce)
}

# Run Bayesian optimization for each model
best_params = {}
trials_dict = {}

for dataset_name, (X_scaled, y_bce) in datasets.items():
    print(f"Optimizing for {dataset_name}...")
    trials = Trials()
    
    best_params[dataset_name] = fmin(fn=lambda params: objective(params, X_scaled, y_bce),
                                     space=space,
                                     algo=tpe.suggest,
                                     max_evals=50,
                                     trials=trials,
                                     rstate=np.random.default_rng(0),  # Seed for reproducibility in hyperopt
                                     early_stop_fn=no_progress_loss(10))
    
    trials_dict[dataset_name] = trials
    print(f"Best hyperparameters for {dataset_name}: {best_params[dataset_name]}")


Optimizing for chemberta...
 20%|██        | 10/50 [00:49<03:19,  4.98s/trial, best loss: 0.33298423886299133]
Best hyperparameters for chemberta: {'dropout_rate': 0.4175299871186762, 'learning_rate': 0.00024043206714908388, 'num_layers': 2.0, 'num_neurons': 180.0}
Optimizing for molformer...
 28%|██▊       | 14/50 [01:19<03:24,  5.69s/trial, best loss: 3.482841968536377]
Best hyperparameters for molformer: {'dropout_rate': 0.22699940019748266, 'learning_rate': 0.006206345322404202, 'num_layers': 2.0, 'num_neurons': 234.0}
Optimizing for molbert...
 32%|███▏      | 16/50 [01:28<03:08,  5.55s/trial, best loss: 1.5918657779693604]
Best hyperparameters for molbert: {'dropout_rate': 0.3476486742192898, 'learning_rate': 0.004207657605324512, 'num_layers': 3.0, 'num_neurons': 256.0}


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import mean_squared_error

# Custom RMSE loss function
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]  # Output layer
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Function to train the model using the best hyperparameters
def train_model(X_train, y_train, best_params):
    # Create DataLoader for training data
    train_dataset = TensorDataset(torch.tensor(X_train.astype(np.float32)), 
                                  torch.tensor(y_train.astype(np.float32)).unsqueeze(1))
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Initialize the model with the best hyperparameters
    model = SimpleNN(input_size=X_train.shape[1],
                     num_layers=int(best_params['num_layers']),
                     num_neurons=int(best_params['num_neurons']),
                     dropout_rate=best_params['dropout_rate'])
    
    # Define the loss and optimizer
    criterion = RMSELoss()
    optimizer = optim.Adam(model.parameters(), lr=best_params['learning_rate'])

    # Train the model
    model.train()
    for epoch in range(100):  # Training for 100 epochs
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    return model

# Load your best hyperparameters from the optimization step
# Replace these with the actual best hyperparameters you found
best_params_chemberta = best_params['chemberta']
best_params_molformer = best_params['molformer']
best_params_molbert = best_params['molbert']

# Train the models for each dataset
best_model_chemberta = train_model(chemberta_X_scaled, chemberta_y_bce, best_params_chemberta)
best_model_molformer = train_model(molformer_X_scaled, molformer_y_bce, best_params_molformer)
best_model_molbert = train_model(molbert_X_scaled, molbert_y_bce, best_params_molbert)

In [8]:
# identify missing in test
missing_smiles_molformer_test = set(bbbp_chemberta2_test['smiles']) - set(bbbp_molformer_test['smiles'])
print(f"Missing smiles in molformer_test: {missing_smiles_molformer_test}")

missing_smiles_molbert_test = set(bbbp_chemberta2_test['smiles']) - set(bbbp_molbert_test['smiles'])
print(f"Missing smiles in molbert_test: {missing_smiles_molbert_test}")

# Combine the invalid smiles from molformer_test with the missing smiles from molbert_test
combined_invalid_smiles_test = list(set(missing_smiles_molformer_test).union(set(missing_smiles_molbert_test)))
print(f"These indices will be removed from the test set: {combined_invalid_smiles_test}")

Missing smiles in molformer_test: {'n1c(csc1\\[NH]=C(\\N)N)c1cccc(c1)NC(C)=O'}
Missing smiles in molbert_test: {'n1c(csc1\\[NH]=C(\\N)N)c1cccc(c1)NC(C)=O'}
These indices will be removed from the test set: ['n1c(csc1\\[NH]=C(\\N)N)c1cccc(c1)NC(C)=O']


In [9]:
# Remove invalid SMILES from each dataframe
bbbp_chemberta2_test = remove_invalid_smiles(bbbp_chemberta2_test, combined_invalid_smiles_test)
bbbp_molformer_test = remove_invalid_smiles(bbbp_molformer_test, combined_invalid_smiles_test)
bbbp_molbert_test = remove_invalid_smiles(bbbp_molbert_test, combined_invalid_smiles_test)
bbbp_chemberta2_features_test = remove_invalid_smiles(bbbp_chemberta2_features_test, combined_invalid_smiles_test)
bbbp_molformer_features_test = remove_invalid_smiles(bbbp_molformer_features_test, combined_invalid_smiles_test)
bbbp_molbert_features_test = remove_invalid_smiles(bbbp_molbert_features_test, combined_invalid_smiles_test)

# Print the shapes of the dataframes after removal
print(bbbp_chemberta2_test.shape)
print(bbbp_molformer_test.shape)
print(bbbp_molbert_test.shape)
print(bbbp_chemberta2_features_test.shape)
print(bbbp_molformer_features_test.shape)
print(bbbp_molbert_features_test.shape)

(204, 8)
(204, 6)
(204, 4)
(204, 386)
(204, 769)
(204, 769)


In [10]:
import numpy as np
from sklearn.metrics import log_loss

# Test data for each model
bbbp_chemberta2_prob_test = pd.DataFrame({'chemberta2': bbbp_chemberta2_test['softmax_class_1_prob']})
bbbp_chemberta2_prob_test.reset_index(drop=True, inplace=True)

bbbp_molformer_prob_test = pd.DataFrame({'molformer': bbbp_molformer_test['Prob_Class_1']})
bbbp_molformer_prob_test.reset_index(drop=True, inplace=True)

bbbp_molbert_prob_test = pd.DataFrame({'molbert': bbbp_molbert_test['prob']})
bbbp_molbert_prob_test.reset_index(drop=True, inplace=True)

bbbp_chemberta2_features_t = pd.DataFrame(bbbp_chemberta2_features_test.iloc[:, 2:])
bbbp_chemberta2_features_t.reset_index(drop=True, inplace=True)

bbbp_molformer_features_t  = pd.DataFrame(bbbp_molformer_features_test.iloc[:, 1:])
bbbp_molformer_features_t.reset_index(drop=True, inplace=True)

bbbp_molbert_features_t = pd.DataFrame(bbbp_molbert_features_test.iloc[:, 1:])
bbbp_molbert_features_t.reset_index(drop=True, inplace=True)

# Combine probabilities with the respective feature sets for the test set
chemberta_X_test = pd.concat([bbbp_chemberta2_prob_test, bbbp_chemberta2_features_t], axis=1)
molformer_X_test = pd.concat([bbbp_molformer_prob_test, bbbp_molformer_features_t], axis=1)
molbert_X_test = pd.concat([bbbp_molbert_prob_test, bbbp_molbert_features_t], axis=1)

# Standardize the test set based on the previously fitted scalers
chemberta_X_test_scaled = scaler_chemberta.transform(chemberta_X_test)
molformer_X_test_scaled = scaler_molformer.transform(molformer_X_test)
molbert_X_test_scaled = scaler_molbert.transform(molbert_X_test)

# Ensure that X_scaled and y_bce are numpy arrays
chemberta_X_test_scaled= np.array(chemberta_X_test_scaled)
molformer_X_test_scaled = np.array(molformer_X_test_scaled)
molbert_X_test_scaled = np.array(molbert_X_test_scaled)

# Predict on the test sets (assuming your test sets are already preprocessed and scaled)
with torch.no_grad():
    chemberta_X_test_tensor = torch.tensor(chemberta_X_test_scaled.astype(np.float32))
    molformer_X_test_tensor = torch.tensor(molformer_X_test_scaled.astype(np.float32))
    molbert_X_test_tensor = torch.tensor(molbert_X_test_scaled.astype(np.float32))
    
    chemberta_pred_test = best_model_chemberta(chemberta_X_test_tensor).numpy()
    molformer_pred_test = best_model_molformer(molformer_X_test_tensor).numpy()
    molbert_pred_test = best_model_molbert(molbert_X_test_tensor).numpy()
        
    # Ensure the predictions are 1-dimensional by flattening if necessary
    chemberta_pred_test = chemberta_pred_test.flatten()
    molformer_pred_test = molformer_pred_test.flatten()
    molbert_pred_test = molbert_pred_test.flatten()
    
# The predictions are now stored in chemberta_pred_test, molformer_pred_test, molbert_pred_test

# Convert the predictions (numpy arrays) to pandas Series
chemberta_pred_test_series = pd.Series(chemberta_pred_test, name='bce_chemberta')
molformer_pred_test_series = pd.Series(molformer_pred_test, name='bce_molformer')
molbert_pred_test_series = pd.Series(molbert_pred_test, name='bce_molbert')

# Now concatenate the series with the test set probabilities
bbbp_X_ensemble_test = pd.concat([
    chemberta_pred_test_series,                     # BCE for Chemberta
    molformer_pred_test_series,                     # BCE for Molformer
    molbert_pred_test_series,                       # BCE for Molbert
    bbbp_chemberta2_prob_test['chemberta2'],        # Chemberta test probabilities
    bbbp_molformer_prob_test['molformer'],          # Molformer test probabilities
    bbbp_molbert_prob_test['molbert']               # Molbert test probabilities
], axis=1)

bbbp_X_ensemble_test.columns = ['bce_chemberta', 'bce_molformer', 'bce_molbert', 'chemberta2', 'molformer', 'molbert']

# optional for evaluation
bbbp_y_ensemble_test = bbbp_chemberta2_test['p_np']

In [11]:
# use standard scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
bbbp_X_ensemble_valid2_scaled = scaler.fit_transform(bbbp_X_ensemble_valid2)
bbbp_X_ensemble_test_scaled = scaler.transform(bbbp_X_ensemble_test)

# transform back to dataframe
bbbp_X_ensemble_valid2_scaled = pd.DataFrame(bbbp_X_ensemble_valid2_scaled, columns=bbbp_X_ensemble_valid2.columns)
bbbp_X_ensemble_test_scaled = pd.DataFrame(bbbp_X_ensemble_test_scaled, columns=bbbp_X_ensemble_test.columns)


In [12]:
bbbp_X_ensemble_valid2_selected = bbbp_X_ensemble_valid2_scaled
bbbp_X_ensemble_test_selected = bbbp_X_ensemble_test_scaled

# check shapes
print(bbbp_X_ensemble_valid2_selected.shape)
print(bbbp_X_ensemble_test_selected.shape)

(401, 6)
(204, 6)


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Set seeds for reproducibility
np.random.seed(0)
torch.manual_seed(0)

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1), nn.Sigmoid()]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Objective function for Bayesian optimization
def objective(params):
    kf = KFold(n_splits=5)
    roc_aucs = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        train_dataset = TensorDataset(torch.tensor(X_train.values.astype(np.float32)), 
                                      torch.tensor(y_train.values.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=int(params['num_layers']), 
                         num_neurons=int(params['num_neurons']), dropout_rate=params['dropout_rate'])
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val.values.astype(np.float32))
            y_val_tensor = torch.tensor(y_val.values.astype(np.float32)).unsqueeze(-1)
            outputs = model(X_val_tensor)
            roc_auc = roc_auc_score(y_val_tensor.numpy(), outputs.numpy())
            roc_aucs.append(roc_auc)

    avg_roc_auc = np.mean(roc_aucs)
    return {'loss': -avg_roc_auc, 'status': STATUS_OK}  # Maximize ROC AUC by minimizing the negative ROC AUC

# Hyperparameter space
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

X = bbbp_X_ensemble_valid2_selected
y = bbbp_y_ensemble_valid2

# Run Bayesian optimization
trials = Trials()
bbbp_nn_best_params = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials,
            rstate=np.random.default_rng(0),  # Seed for hyperopt
            early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", bbbp_nn_best_params)


 20%|██        | 10/50 [00:35<02:21,  3.53s/trial, best loss: -1.0]
Best hyperparameters: {'dropout_rate': 0.4175299871186762, 'learning_rate': 0.00024043206714908388, 'num_layers': 2.0, 'num_neurons': 180.0}


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import roc_auc_score

torch.manual_seed(0)

# Define the neural network model again
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1), nn.Sigmoid()]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Convert parameters to the correct format if necessary
bbbp_nn_best_params = {
    'num_layers': int(bbbp_nn_best_params['num_layers']),  # Extracted from Bayesian optimization results
    'num_neurons': int(bbbp_nn_best_params['num_neurons']),  # Extracted from Bayesian optimization results
    'dropout_rate': bbbp_nn_best_params['dropout_rate'],  # Extracted from Bayesian optimization results
    'learning_rate': bbbp_nn_best_params['learning_rate']  # Extracted from Bayesian optimization results
}

# Prepare datasets
X_train_tensor = torch.tensor(bbbp_X_ensemble_valid2_selected.values.astype(np.float32))
y_train_tensor = torch.tensor(bbbp_y_ensemble_valid2.values.astype(np.float32)).unsqueeze(1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_test_tensor = torch.tensor(bbbp_X_ensemble_test_selected.values.astype(np.float32))
y_test_tensor = torch.tensor(bbbp_y_ensemble_test.values.astype(np.float32)).unsqueeze(1)

# Initialize the model
model = SimpleNN(input_size=bbbp_X_ensemble_valid2_selected.shape[1], num_layers=bbbp_nn_best_params['num_layers'], 
                 num_neurons=bbbp_nn_best_params['num_neurons'], dropout_rate=bbbp_nn_best_params['dropout_rate'])

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=bbbp_nn_best_params['learning_rate'])

# Training loop
model.train()
for epoch in range(100):  # Number of epochs can be adjusted
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluation on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = (outputs > 0.5).float()

    # Calculate metrics
    accuracy = accuracy_score(y_test_tensor.numpy(), predictions.numpy())
    f1 = f1_score(y_test_tensor.numpy(), predictions.numpy())
    roc_auc = roc_auc_score(y_test_tensor.numpy(), outputs.numpy())
    pr_auc = average_precision_score(y_test_tensor.numpy(), outputs.numpy())

    bbbp_nn_metrics = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'ROC-AUC': roc_auc,
        'PR-AUC': pr_auc
    }

bbbp_nn_metrics

{'Accuracy': 0.9019607843137255,
 'F1 Score': 0.9375,
 'ROC-AUC': 0.9586239695624604,
 'PR-AUC': 0.9905798916901642}

In [15]:
# report all the metrics for bbbp
bbbp_metrics_results["Neural Network"] = bbbp_nn_metrics

bbbp_metrics_df = pd.DataFrame(bbbp_metrics_results).T

# keep 3 digits after the decimal point
bbbp_metrics_df = bbbp_metrics_df.round(3)

# export as csv
bbbp_metrics_df.to_csv('./split3_bbbp_metrics_nn.csv')